<a href="https://colab.research.google.com/github/CodingTomo/PyTorch-Tutorials/blob/master/PyTorch_Moduli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Moduli

PyTorch mette a disposizione dello sviluppatore una serie di moduli per la costruzione di reti neurali (NN). Un **modulo** non è altro che una scatoletta che, presi in input dei dati, esegue una determinata operazione. Ciò che esce dalla scatola, come output, è il risultato di questa operazione. Una NN, il più delle volte, è rappresentata da una concatenazione di questi moduli.

I moduli Pytorch consentono facilmente di 
1. tenere traccia dei parametri che li compongono;
2. salvare/caricare moduli già addestrati;
3. azzerare efficacemente i gradienti;
4. spostare in un solo colpo l'elaborazione da CPU a GPU.

In [0]:
import torch

Per prendere dimestichezza con la sintassi, metodi e attributi delle principali entità coinvolte, consideriamo un modulo della classe *nn* che opera una **trasformazione lineare**

$$ y = xA^T+b.$$

A questo scopo, supponiamo di avere un dataset di 15 osservazioni descritte da 5 variabili con associati due valori obiettivo.

In [0]:
X = torch.rand(15,5)
y = torch.rand(15, 2)

linear_trasformation = torch.nn.Linear(5,2) # modulo accetta input di dimensione 5 e genera output di dimensione 2
print('I pesi con cui il modulo è inizializzato sono: \n ', linear_trasformation.weight)
print('____________________________')
print('Il bias con cui il modulo è inizializzato è: \n ', linear_trasformation.bias)

y_hat=linear_trasformation(X)
print('____________________________')
print('I valori predetti per y sono : \n ', y_hat)

In questo esempio, la predizione così come i dati non hanno senso, tuttavia possiamo fingere che lo abbiano e calcolare sia l'errore compiuto sia il suo gradiente rispetto ai parametri della trasformazione.

In [0]:
loss = torch.sum((y - y_hat)**2) # errore quadratico
linear_trasformation.zero_grad() 
loss.backward()

print('Gradiente della funzione di loss rispetto ai pesi: \n ',linear_trasformation.weight.grad)
print('____________________________')
print('Gradiente della funzione di loss rispetto al bias: \n ', linear_trasformation.bias.grad)


Modelli realistici sono in realtà una concatenazione di più moduli. Per fare questo Pytorch mette a disposizione il *container* **Sequential**.

In [0]:
X = torch.rand(15,5)
y = torch.rand(15, 2)

simple_neural_network = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2)
)

y_hat = simple_neural_network(X) # run della rete neurale

#esplorazione dei parametri 
[print("{:8s} shape = {}".format(param, tensor.shape)) for param, tensor in simple_neural_network.named_parameters()]

**Osservazione**: l'indice $1$ non viene stampato perchè il modulo ReLU non ha parametri, ma è l'applicazione statica dell'omonima funzione. 

[Ulteriori informazioni su ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))

Possiamo costruire dei **moduli personalizzati** andando a definire una **classe** che eredita da *torch.nn.Module*. 

Ogni modulo personalizzato deve necessariamente implementare due metodi:

1. *init()* contiene l'inizializzazione della classe padre e l'elenco dei moduli primitivi che andranno a comporre il modulo personalizzato.
2. *forward()* contiene il piano d'esecuzione dei moduli primitivi definiti in *init*.

Di seguito definiamo il modulo personalizzato che implementa *simple_neural_network* dell'esempio precedente.

In [0]:
class MySimpleNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.linear_0 = torch.nn.Linear(input_size, 10)
        self.ReLU = torch.nn.ReLU()
        self.linear_1 = torch.nn.Linear(10, output_size)

    
    def forward(self, x):
        out_0 = self.linear_0(x)
        out_1 = self.ReLU(out_0)
        out_2 = self.linear_1(out_1)
        return out_2

In [0]:
X = torch.rand(15,5)
y = torch.rand(15, 2)

y_hat = simple_neural_network(X) # run della rete neurale

simple_neural_network = MySimpleNeuralNetwork(input_size = 5, output_size = 2)
print('Panoramica generale sul modulo personalizzato: \n',simple_neural_network)
print('____________________________')
[print(name, ":\n", p) for name, p in simple_neural_network.named_parameters()]



### Esempio di classificazione usando il dataset MNIST

Vogliamo **classificare** delle immagini 28x28 contenenti delle cifre manoscritte usando una rete neurale. Useremo il dataset MNIST composto da 60.000 immagini di training e 10.000 immagini di test. Questo dataset è tra quelli di default nella libreria *torchvision* ed è già spezzato opportunamente in *train* e *test*.

<a href="https://ibb.co/NL0ZwLv"><img src="https://i.ibb.co/XznSmzQ/mnist.jpg" alt="mnist" border="0"></a>

In [0]:
import torchvision
import torch
import numpy

# MNIST Dataset (Images and Labels)
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False, # spicifica che ci serve la parte di test
    transform=torchvision.transforms.ToTensor()
)

In [0]:
print('Il numero di immagini nel train è {}, mentre quello nel test è {}'.format(len(train_dataset),len(test_dataset)))

**Osservazione**: se eseguiamo il comando 
`
train_dataset[0]
`
otteniamo la prima coppia del dataset contenente l'immagine e la relativa etichetta (intero da 0 a 9). Se invece scriviamo
`
train_dataset[0][0].shape
`
otteniamo la dimensione tensoriale dell'immagine.



Esercizio: Riflettere sul l'output del comando `train_dataset[0][0].shape`. Cosa cambierebbe se l'immagine fosse a colori? 

Quando addetriamo un modello sono necessarie molte iterazioni sull'intero dataset di train. Ogni volta le osservazioni vengono mescolate e impacchettate in sottoinsiemi, detti *batch*, più o meno grossi. Questa operazione è presa in carico dal **DataLoader** di PyTorch.

In [0]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Possiamo controllare la bontà del nostro operato eseguendo degli **assert** nel modo seguente.

In [0]:
for images, labels in train_loader:
    assert len(images) == batch_size
    assert len(labels) == batch_size

**Osservazione**: DataLoader supporta il *multi-threading* specificando l'attributo *num_workers*.


```
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=?)
```



Una **rete neurale** è formata da un insieme di strati consecutivi, detti *layer*, ognuno dei quali è composto da neuroni. Ogni neurone è connesso con tutti i neuroni dello strato immediatamente precedente e immediatamente successivo. Il **primo strato** è costruito per prendere ingresso il dataset di input e ha un numero neuroni pari al numero di variabili che descrivono un'ossevazione. L'**ultimo strato** invece genera la predizione. Per ora, pensiamo ai **layer intermedi** come a delle trasformazioni utili per analizzare nel dettaglio ogni singolo input. I parametri da addestrare in una rete neurale sono i **pesi** con cui i neuroni sono interconnessi fra loro.

Nel nostro esempio il primo layer accetterà vettori di lunghezza 28x28, mentre l'ultimo genererà, per ogni vettore di input, un vettore di lunghezza $10$, dove la coodinata con valore più grande sarà l'etichetta prevista dalla rete per quella particolare istanza di input.

<a href="https://ibb.co/GC7Znzc"><img src="https://i.ibb.co/qmBH7Tk/neural-net2.jpg" alt="neural-net2" border="0"></a>

Per assicurarci che la rete neurale sia in grado cogliere relazioni non lineari fra le osservazioni, ogni output di ogni neurone viene trasformato tramite un funzione non lineare $\sigma(\cdot)$ detta **activation function**.

Più precisamente, i neuroni $\vec x_{i+1}$ del layer $i+1$ sono "calcolati" a partire dai neuroni $\vec x_i$ del layer $i$ come 

$$ \vec x_{i+1} = \sigma\left(W_{i+1} \vec x_i + \vec b_{i+1} \right) $$

dove $W_{i+1}$ sono i pesi della rete per ogni coppia di neuroni input/output nello strato $i+1$ e con $\vec b_{i+1}$ il termine di bias. Osserviamo che $\sigma$ opera elemento per elemento.

Andiamo quindi a **definire** il nostro classificatore.

In [0]:
import torch.nn.functional as funct

class MyClassifier(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyClassifier, self).__init__()
        
        self.input_size = input_size
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(input_size, 75)
        self.linear_2 = torch.nn.Linear(75, 50)
        self.linear_3 = torch.nn.Linear(50, num_classes)
        
    
    def forward(self, x):
        out = funct.relu(self.linear_1(x))
        out = funct.relu(self.linear_2(out))
        out = self.linear_3(out)
        return out

Per verificare che il modulo sia ben definito proviamolo con un esempio casuale.

In [0]:
x = torch.rand(1, 28 * 28) 
model = MyClassifier(input_size=28 * 28, num_classes=10)
out = model(x)

Costruiamo una funzione che ci aiuterà a capire le **performance** del nostro modello. In questo caso ci concetriamo sull'**accuratezza**, cioè il rapporto fra il numero di esempi etichettati correttamente dal modello e il numero totale degli esempi.

In [0]:
def accuracy(model, data_loader, device):
    with torch.no_grad(): # non ci servono i gradienti nel test. Il contesto no_grad() velocizza il processo.
        correct = 0
        total = 0
        for inputs, labels in data_loader:

            inputs = inputs.to(device)

            # Preprocessiamo le immagini in modo che risultino dei vettori di dimensione 1 e lunghezza 28x28     
            inputs = inputs.view(-1, 28*28)
            
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            correct += (predicted.cpu() == labels).sum().item()
            total += labels.size(0)
            
    acc = correct / total
    return acc

Ora **addestriamo** il nostro classificatore.

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Copia di tutti i parametri del modello nella device corrente
model = model.to(device)
acc = accuracy(model, test_loader, device)
print('La precisione del modello con pesi non addestrati è del {0:.0%}'.format(acc))
print('____________________________')

# Definiamo funzione di errore e ottimizzatore passandogli tutti i parametri del modello addestrare 
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

time = numpy.array([])
avg_loss = numpy.array([])
acc_list = numpy.array([])

# Iteriamo sull'intero dataset per 5 volte
for epoch in range(15):
    total_loss = 0.0
    
    # Iteriamo sui batch precedentemente costruiti 
    for (inputs, labels) in train_loader:
        
        # Copia dei dati necessari nella device corrente
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Preprocessiamo le immagini in modo che risultino dei vettori di dimensione 1 e lunghezza 28x28
        inputs = inputs.view(-1, 28*28)

        # Per ogni batch facciamo calcoliamo la predizione, l'errore e ottimizziamo
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Sommiamo l'errore di ogni batch
        total_loss += loss.item()
    
    time = numpy.append(time,[epoch+1])
    avg_loss = numpy.append(avg_loss,[total_loss/len(train_loader)])

    acc = accuracy(model, test_loader, device)
    acc_list = numpy.append(acc_list,[acc])

    # Media dell'errore per epoca    
    print('La funzione di errore dopo epoca numero {0} è del {1}'.format(epoch+1, total_loss/len(train_loader))) 
    print('La precisione del modello dopo epoca numero {0} è del {1:.0%}'.format(epoch+1,acc))
    print('____________________________')

**Osservazione**: in fase di addestrmento abbiamo scelto una particolare funzione di errore. La `CrossEntropyLoss` scelta non è altro che una versione particolare della **funzione di verosimiglianza**.

Di seguito una lista non esaustiva di possibili alternative.
- `L1Loss`
- `MSELoss`
- `CrossEntropyLoss`
- `NLLLoss`
- `PoissonNLLLoss`
- `KLDivLoss`
- `BCELoss`
- `...`

[Ulteriori informazioni sulla funzione di verosimiglianza](https://en.wikipedia.org/wiki/Likelihood_function)

In [0]:
from matplotlib import pyplot as plt

def plot(x_axis,x_label,y_axis,y_label,curve_label):
   plt.plot(x_axis, y_axis, label=curve_label)
   plt.xlabel(x_label)
   plt.ylabel(y_label)
   plt.legend(shadow=True)

In [0]:
plot(x_axis=time,x_label="epoch",y_axis=avg_loss,y_label="average loss",curve_label="Training loss")

In [0]:
plot(x_axis=time,x_label="epoch",y_axis=acc_list,y_label="accuracy",curve_label="Test accuracy")

**Esercizio**: Aggiungere al precedente grafico l'accuratezza del training set rispetto alle epoche compiute.
```
Suggerimento: accuracy(model, train_loader, device)
```



In contesti reali è fondamentale poter **salvare** i risutati ottenuti e ciò significa essere in grado di salvare il modello che quei risultati gli ha ottenuti. Nella pratica questo si traduce nel salvare i pesi già addestrati del modello ed eventualmente ricaricarli in seguito.

In [0]:
torch.save(model, "*PATH*\my_classifier.pt")
model = torch.load("*PATH*\my_classifier.pt")